# Quick and dirty MOC

## part of a project to quantify transports across the Greenland-Scotland section by source and destination

- using output from Ocean Parcels, but just the initial particle properties
- random positioned particles with T,S and velocity components
- using these to diagnose transports by density and velocity bin

## technical preamble

In [ ]:
%matplotlib inline

In [ ]:


from pathlib import Path
import numpy as np
import xarray as xr
from datetime import datetime, timedelta
import seaborn as sns
import cmocean as co
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seawater as sw
from matplotlib import rcParams
rcParams['mathtext.default'] = 'regular'
np.warnings.filterwarnings('ignore')


## input file parameters

In [ ]:
def set_file_names(year,mon):
    
    year_str = str(year)
    mon_str = str(mon).zfill(2)

    path_data_tracks = {}
    filename = {}

    # Project path
    project_path = Path.cwd() / '..' / '..' 
    project_path = project_path.resolve()

    # Faroe-Shetland
    path_data_tracks['FS'] = Path('data/processed/tracks/FaroeShetland/') 
    filename['FS'] = 'tracks_FS_backward_'+year_str+mon_str+'_N4000_D20_Rnd123.nc'
    # Iceland-Faroe
    path_data_tracks['IF'] = Path('data/processed/tracks/IcelandFaroe/') 
    filename['IF'] = 'tracks_IF_backward_'+year_str+mon_str+'_N4000_D20_Rnd123.nc'
    # Greenland-Iceland
    path_data_tracks['GI'] = Path('data/processed/tracks/GreenlandIceland/') 
    filename['GI'] = 'tracks_GI_backward_'+year_str+mon_str+'_N4000_D20_Rnd123.nc'
    
    return project_path,path_data_tracks,filename


## velocity conversion parameters

In [ ]:
def velocity_conversion_params():
    lat2metres = {}
    lon2metres = {}

    sealandratio = {}
    sectionAreaEW = {}
    sectionAreaNS = {}
    sectionArea = {}
    sectionAngle = {}

    #velocity conversions on FS channel (61N)

    lat2metres['FS'] = 60.0*1852.0
    lon2metres['FS'] = 60.0*1852.0*np.cos(np.radians(61.0))

    sealandratio['FS'] = 5591.0/20000.0
    sectionAreaEW['FS'] = 5.7 * 1500.0 * sealandratio['FS'] * lon2metres['FS']  # very approx. mean depth x length between slopes.
    sectionAreaNS['FS'] = 1.87 * 1500.0 * sealandratio['FS'] * lat2metres['FS']
    sectionArea['FS'] = np.sqrt(sectionAreaEW['FS']**2 + sectionAreaNS['FS']**2)
    sectionAngle['FS'] = np.arctan2(1.87 * lat2metres['FS'], 5.7 * lon2metres['FS'])


    #velocity conversions on IFR (63.5N)

    lat2metres['IF'] = 60.0*1852.0
    lon2metres['IF'] = 60.0*1852.0*np.cos(np.radians(63.5))

    sealandratio['IF'] = 9308.0/20000.0
    sectionAreaEW['IF'] = 7.0 * 700.0 * sealandratio['IF'] * lon2metres['IF']  # very approx. mean depth x length between slopes.
    sectionAreaNS['IF'] = 2.8 * 700.0 * sealandratio['IF'] * lat2metres['IF']
    sectionArea['IF'] = np.sqrt(sectionAreaEW['IF']**2 + sectionAreaNS['IF']**2)
    sectionAngle['IF'] = np.arctan2(1.87 * lat2metres['IF'], 5.7 * lon2metres['IF'])



    #velocity conversions on GI (67.3N)
    lat2metres['GI'] = 60.0*1852.0
    lon2metres['GI'] = 60.0*1852.0*np.cos(np.radians(67.3))

    sealandratio['GI'] = 2420.0/10000.0
    sectionAreaEW['GI'] = 5.73 * 1500.0 * sealandratio['GI'] * lon2metres['GI']  # very approx. mean depth x length between slopes.
    sectionAreaNS['GI'] = 2.94 * 1500.0 * sealandratio['GI'] * lat2metres['GI']
    sectionArea['GI'] = np.sqrt(sectionAreaEW['GI']**2 + sectionAreaNS['GI']**2)
    sectionAngle['GI'] = np.arctan2(1.87 * lat2metres['GI'], 5.7 * lon2metres['GI'])
    
    return     lat2metres, lon2metres,sealandratio,sectionAreaEW,sectionAreaNS,sectionArea,sectionAngle

lat2metres, lon2metres,sealandratio,sectionAreaEW,sectionAreaNS,sectionArea,sectionAngle = velocity_conversion_params()

## helper functions

In [ ]:
def transports(dsb, nParticles,section):
    transport = (sectionAreaEW[section]*dsb.isel(obs=0).vvel*lat2metres[section] +
                            sectionAreaNS[section]*dsb.isel(obs=0).uvel*lon2metres[section])/nParticles
    speed = np.sqrt((dsb.isel(obs=0).vvel*lat2metres[section])**2 + (dsb.isel(obs=0).uvel*lon2metres[section])**2)
    return transport,speed

def normalVelocitiesT0(dsb, angle,section):
    return dsb.isel(obs=0).vvel*lat2metres[section]*np.cos(angle) + dsb.isel(obs=0).uvel*lon2metres[section]*np.sin(angle)

In [ ]:
fig,ax = plt.subplots(3,2,figsize=(10,14))

plt.setp(ax, xlim=(-7,7), ylim=(28.1,26))
# Hide x labels and tick labels for top plots and y ticks for right plots.
for ii,month in enumerate([3,5,7,8,10,12]):
# for ii,month in enumerate([3]):
    
    irow = ii // 2
    icol = ii % 2

    project_path,path_data_tracks,filename = set_file_names(2014,month)

    ### read in track data

    dsfs = xr.open_dataset(project_path / path_data_tracks['FS'] / filename['FS'])
    dsfs.temp.attrs['standard_name'] = 'potential temperature'
    dsfs.temp.attrs['units'] = 'degree_C'
    dsfs.salt.attrs['standard_name'] = 'practical salinity'
    dsfs.salt.attrs['units'] = 'PSU'
    dsif = xr.open_dataset(project_path / path_data_tracks['IF'] / filename['IF'])
    dsif.temp.attrs['standard_name'] = 'potential temperature'
    dsif.temp.attrs['units'] = 'degree_C'
    dsif.salt.attrs['standard_name'] = 'practical salinity'
    dsif.salt.attrs['units'] = 'PSU'
    dsgi = xr.open_dataset(project_path / path_data_tracks['GI'] / filename['GI'])
    dsgi.temp.attrs['standard_name'] = 'potential temperature'
    dsgi.temp.attrs['units'] = 'degree_C'
    dsgi.salt.attrs['standard_name'] = 'practical salinity'
    dsgi.salt.attrs['units'] = 'PSU'


    ## Calculate some basic parameters from data for later

    nParticles = {}
    nobsb = {}

    nParticles['FS'] = dsif.sizes['traj']
    nobsb['FS'] = dsif.sizes['obs']
    nParticles['IF'] = dsif.sizes['traj']
    nobsb['IF'] = dsif.sizes['obs']
    nParticles['GI'] = dsif.sizes['traj']
    nobsb['GI'] = dsif.sizes['obs']

    # calculate initial transports

    particle_transport_fs,speed0_fs = transports(dsfs,nParticles['FS'],'FS')
    normal_velocity_fs = normalVelocitiesT0(dsfs,sectionAngle['FS'],'FS')
    particle_transport_if,speed0_if = transports(dsif,nParticles['IF'],'IF')
    normal_velocity_if = normalVelocitiesT0(dsif,sectionAngle['IF'],'IF')
    particle_transport_gi,speed0_gi = transports(dsgi,nParticles['GI'],'GI')
    normal_velocity_gi = normalVelocitiesT0(dsgi,sectionAngle['GI'],'GI')


    ## calculate potential densities of particles at T0

    # there are neater ways to do this with xarray if the datasets are large

    S_fs = dsfs.isel(obs=0).salt
    T_fs = dsfs.isel(obs=0).temp
    sig0_fs = sw.pden(S_fs,T_fs,0.0,0.0)
    sig0_fs -= 1000.0

    S_if = dsif.isel(obs=0).salt
    T_if = dsif.isel(obs=0).temp
    sig0_if = sw.pden(S_if,T_if,0.0,0.0)
    sig0_if -= 1000.0

    S_gi = dsgi.isel(obs=0).salt
    T_gi = dsgi.isel(obs=0).temp
    sig0_gi = sw.pden(S_gi,T_gi,0.0,0.0)
    sig0_gi -= 1000.0


    ## construct density vs. velocity transport matrix


    transportGrid_fs = np.zeros((260,200))
    transportGrid_if = np.zeros((260,200))
    transportGrid_gi = np.zeros((260,200))

    x = np.arange(25.5,28.1001,0.01)
    y = np.arange(-1.0,1.0001,0.01)
    x1 = np.arange(25.5,28.0999,0.01)

    xc=np.arange(25.505,28.100,0.01)
    yc = np.arange(-0.995,1.000,0.01)


    for i in range(nParticles['FS']):
        transportGrid_fs[np.floor((sig0_fs[i]-25.5)*100.0).astype(int),np.floor((normal_velocity_fs[i]+1.0)*100.0).astype(int)] += particle_transport_fs[i]/1.0e06
    for i in range(nParticles['IF']):
        transportGrid_if[np.floor((sig0_if[i]-25.5)*100.0).astype(int),np.floor((normal_velocity_if[i]+1.0)*100.0).astype(int)] += particle_transport_if[i]/1.0e06
    for i in range(nParticles['GI']):
        transportGrid_gi[np.floor((sig0_gi[i]-25.5)*100.0).astype(int),np.floor((normal_velocity_gi[i]+1.0)*100.0).astype(int)] += particle_transport_gi[i]/1.0e06


    tGxr_fs = xr.DataArray(data=transportGrid_fs,coords={'sig0':xc,'velocity':yc},dims=['sig0','velocity'],name='Transport')

    tGxr_fs.attrs['units'] = r"Sv"
    tGxr_fs.sig0.attrs['units'] = r"$kg\ m^{-3}$"
    tGxr_fs.velocity.attrs['units'] = r"$m\ s^{-1}$"

    tGxr_if = xr.DataArray(data=transportGrid_if,coords={'sig0':xc,'velocity':yc},dims=['sig0','velocity'],name='Transport')

    tGxr_if.attrs['units'] = r"Sv"
    tGxr_if.sig0.attrs['units'] = r"$kg\ m^{-3}$"
    tGxr_if.velocity.attrs['units'] = r"$m\ s^{-1}$"

    tGxr_gi = xr.DataArray(data=transportGrid_gi,coords={'sig0':xc,'velocity':yc},dims=['sig0','velocity'],name='Transport')

    tGxr_gi.attrs['units'] = r"Sv"
    tGxr_gi.sig0.attrs['units'] = r"$kg\ m^{-3}$"
    tGxr_gi.velocity.attrs['units'] = r"$m\ s^{-1}$"


    ## contruct MOC plot

    tGxr = tGxr_fs + tGxr_if + tGxr_gi
    tGxr.attrs['units'] = r"Sv"

    tGxrcumsum_fs = tGxr_fs.where(tGxr_fs != 0.0).sum(dim='velocity').cumsum()
    tGxrcumsum_fs.attrs['units']=r"Sv"
    tGxrcumsum_fs.name=r"Cumulative Transport from low-high density"

    tGxrcumsum_if = tGxr_if.where(tGxr_if != 0.0).sum(dim='velocity').cumsum()
    tGxrcumsum_if.attrs['units']=r"Sv"
    tGxrcumsum_if.name=r"Cumulative Transport from low-high density"

    tGxrcumsum_gi = tGxr_gi.where(tGxr_gi != 0.0).sum(dim='velocity').cumsum()
    tGxrcumsum_gi.attrs['units']=r"Sv"
    tGxrcumsum_gi.name=r"Cumulative Transport from low-high density"

    tGxrcumsum = tGxr.where(tGxr != 0.0).sum(dim='velocity').cumsum()
    tGxrcumsum.attrs['units']=r"Sv"
    tGxrcumsum.name=r"Cumulative Transport from low-high density"

    tGxrcumsum_fs.plot(y='sig0',label='FS',ax=ax[irow,icol])
    tGxrcumsum_if.plot(y='sig0',label='IF',ax=ax[irow,icol])
    tGxrcumsum_gi.plot(y='sig0',label='GI',ax=ax[irow,icol])
    tGxrcumsum.plot(y='sig0',label='Total',ax=ax[irow,icol])

    ax[irow,icol].legend()
#    ax[irow,icol].set_ylim((28.1,26.0))
#    ax[irow,icol].set_ylim((-7,7))

plt.tight_layout()
    